In [2]:
import pandas as pd
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
from decouple import Config, RepositoryEnv


In [3]:
# Load the environment variables using python-decouple
# The .env file should be in the root of the project
# The .env file should NOT be committed to the repository

config = Config(RepositoryEnv(".env.paul"))

# Load the credentials
WX_API_KEY = config("WX_API_KEY")

In [4]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)
client = APIClient(
    credentials=credentials, 
    project_id="68126b74-155e-4a70-aa2c-1781dfad87f6"
)

In [19]:
# --- Setup the Model (Granite-13b-chat-v2) ---
params = TextGenParameters(
    temperature=0.2,            # Light creativity
    max_new_tokens=3000          # Enough for descriptive text
)

model = ModelInference(
    api_client=client,
    params=params,
    model_id="ibm/granite-3-8b-instruct",
)
prompt = "Hello from copenhagen!"
generated_response = model.generate(prompt)

generated_response

{'model_id': 'ibm/granite-3-8b-instruct',
 'model_version': '1.1.0',
 'created_at': '2025-04-29T11:42:50.710Z',
 'results': [{'generated_text': "\n\nI'm currently in Copenhagen, Denmark, for a conference. It's a beautiful city with a rich history and a vibrant cultural scene. I've been exploring the city, visiting iconic landmarks, and enjoying the local cuisine.\n\nOne of the highlights of my trip so far has been visiting the Little Mermaid statue, a beloved symbol of Copenhagen. The statue, based on the fairy tale by Hans Christian Andersen, is a must-see for anyone visiting the city.\n\nI've also been indulging in Danish pastries, such as the famous kanelsnegl (cinnamon swirls) and smørrebrød (open-faced sandwiches). The local coffee culture is also something I've come to appreciate, with cozy cafes serving delicious coffee and pastries.\n\nIn addition to sightseeing, I've been attending conference sessions and networking with fellow professionals. It's been an enlightening experien

In [20]:
def create_prompt(sign_name: str) -> str:
    return (
        f"You are an American Sign Language (ASL) teacher.\n\n"
        f"Please clearly explain how to perform the ASL sign on a beginner level for '{sign_name}'. "
        f"Use simple language and full sentences. Do not assume any prior knowledge about ASL.\n\n"
    )


In [21]:
# --- Generate ASL Description ---
def generate_asl_description(sign_name: str) -> str:
    prompt = create_prompt(sign_name)
    
    response = model.generate(prompt=prompt)
    generated_text = response['results'][0]['generated_text']
    
    return generated_text

In [22]:
# --- Example Usage ---
if __name__ == "__main__":
    sign_to_explain = input("📝 Enter the ASL sign you want explained: ")

    try:
        description = generate_asl_description(sign_to_explain)
        print("\n✅ Generated ASL Description:\n")
        print(description)
    except Exception as e:
        print(f"❌ Error during generation: {e}")


✅ Generated ASL Description:


To sign 'laptop' in American Sign Language (ASL), follow these steps:

1. Start with your dominant hand open and facing up, palm facing you.
2. Bring your hand down to your lap, as if you're placing an object there.
3. With your hand still in your lap, bend your index finger and middle finger down to touch the tips of your thumb, forming a 'C' shape.
4. Keep your ring finger and pinky finger straight and slightly apart.
5. Maintain this 'C' shape with your fingers while keeping your hand in your lap.
6. To indicate that it's a laptop, you can add a small, quick, circular motion with your hand, as if you're turning on a device.

Remember, practice makes perfect. Keep practicing this sign until you feel comfortable with it.


Judge

In [31]:
def create_judge_prompt(sign_name: str, description: str) -> str:
    return f"""You are evaluating the quality of a generated ASL (American Sign Language) sign description based on 3 criteria:

1. Faithfulness: Is the description factually consistent with known ASL practices (no hallucinations)?
2. Answer Relevance: Does the description directly and completely describe the sign requested?
3. Context Relevance: Would a typical ASL reference source support this description for the given sign?

Use the following scale:
- 1 = fully meets the criterion
- 0.5 = partially meets the criterion
- 0 = does not meet the criterion

ASL Sign Request:
"{sign_name}"

Generated Description:
"{description}"

Respond as a JSON object, exactly like this (no extra text or comments):

{{
  "faithfulness": <0, 0.5, or 1>,
  "answer_relevance": <0, 0.5, or 1>,
  "context_relevance": <0, 0.5, or 1>
}}"""









In [32]:
def evaluate_description(sign_name: str, description: str) -> dict:
    import json, re

    judge_prompt = create_judge_prompt(sign_name, description)
    response = model.generate(prompt=judge_prompt)
    judged_text = response['results'][0]['generated_text']

    # Default fallback result
    judge_result = {
        "faithfulness": None,
        "answer_relevance": None,
        "context_relevance": None
    }

    try:
        parsed = json.loads(judged_text)
        if isinstance(parsed, dict):
            judge_result.update(parsed)
    except Exception:
        matches = re.findall(r"\{.*?\}", judged_text, re.DOTALL)
        if matches:
            try:
                parsed = json.loads(matches[0])
                if isinstance(parsed, dict):
                    judge_result.update(parsed)
            except Exception as e:
                print(f"❌ Fallback parse failed: {e}")
        else:
            print("❌ No JSON object found in model output.")

    return judge_result





In [36]:
if __name__ == "__main__":
    print("👋 Welcome to the ASL Sign Generator using Granite-13b!")
    sign_to_explain = input("📝 Enter the ASL sign you want explained: ")

    try:
        # Generate ASL description
        description = generate_asl_description(sign_to_explain)
        print("\n✅ Generated ASL Description:\n")
        print(description)

        # Judge the description
        judge_result = evaluate_description(sign_to_explain, description)

        print("\n📊 Evaluation Result:")
        print(f" Faithfulness: {judge_result.get('faithfulness')}")
        print(f" Answer Relevance: {judge_result.get('answer_relevance')}")
        print(f" Context Relevance: {judge_result.get('context_relevance')}")

    except Exception as e:
        print(f"❌ Error during generation or evaluation: {e}")





👋 Welcome to the ASL Sign Generator using Granite-13b!

✅ Generated ASL Description:


To sign 'car' in American Sign Language (ASL), follow these steps:

1. Start with your dominant hand open and facing up, representing the general concept of a vehicle.
2. Bring your hand down to your waist level, moving it in a small circular motion to indicate the circular shape of a car's body.
3. Extend your index finger and middle finger, keeping them together, to represent the car's windshield.
4. Bend your index finger slightly to show the curve of the windshield.
5. Move your hand forward and slightly to the right, as if you're pointing to a car parked in front of you.

Remember, ASL is a visual language, so practice the motion and rhythm to ensure you're signing 'car' accurately. Keep practicing, and you'll become more comfortable with the sign.

📊 Evaluation Result:
 Faithfulness: 1
 Answer Relevance: 1
 Context Relevance: 1


Mass evaluation

In [39]:
import pandas as pd

def evaluate_batch(signs: list[str], reference_definitions: list[str]) -> pd.DataFrame:
    """
    Evaluate multiple ASL signs using the generate + judge pipeline.
    Returns a DataFrame with individual scores and definitions.
    """
    results = []

    for sign, expected in zip(signs, reference_definitions):
        try:
            # Step 1: Generate ASL description
            generated = generate_asl_description(sign)

            # Step 2: Evaluate the description
            scores = evaluate_description(sign, generated)

        except Exception as e:
            print(f"❌ Error for sign '{sign}': {e}")
            generated = "ERROR"
            scores = {
                "faithfulness": None,
                "answer_relevance": None,
                "context_relevance": None
            }

        results.append({
            "Sign": sign,
            "Expected Definition": expected,
            "Generated Description": generated,
            "Faithfulness": scores.get("faithfulness"),
            "Answer Relevance": scores.get("answer_relevance"),
            "Context Relevance": scores.get("context_relevance")
        })

    df = pd.DataFrame(results)

    # Print the full table
    print("\n📋 Evaluation Table:\n")
    print(df)

    # Print average scores
    avg_faithfulness = df["Faithfulness"].dropna().mean()
    avg_relevance = df["Answer Relevance"].dropna().mean()
    avg_context = df["Context Relevance"].dropna().mean()

    print("\n📊 Average Scores:")
    print(f"Faithfulness: {avg_faithfulness:.2f}")
    print(f"Answer Relevance: {avg_relevance:.2f}")
    print(f"Context Relevance: {avg_context:.2f}")

    return df


In [43]:
signs = ["salad", "pizza"]
definitions = [
    "Mimic the motion of tossing a salad by placing both hands next to each other in front of your with your palms oriented toward each other, and then slightly moving your hands toward each other and up a couple of times.",
    "Place your hand in front of your face with your fingers pointing at your mouth. Then, slightly move your hand toward your mouth a couple of times."
]


In [45]:
# Step 1: Evaluate all signs
df = evaluate_batch(signs, definitions)

# Step 2: Calculate average scores
scores = {
    "faithfulness": df["Faithfulness"].dropna().mean(),
    "answer_relevance": df["Answer Relevance"].dropna().mean(),
    "context_relevance": df["Context Relevance"].dropna().mean()
}

# Step 3: Plot the result
plot_judge_scores(scores)






📋 Evaluation Table:

    Sign                                Expected Definition  \
0  salad  Mimic the motion of tossing a salad by placing...   
1  pizza  Place your hand in front of your face with you...   

                               Generated Description  Faithfulness  \
0  \nTo sign 'salad' in American Sign Language (A...             1   
1  \nTo sign 'pizza' in American Sign Language (A...             1   

   Answer Relevance  Context Relevance  
0                 1                  1  
1                 1                  1  

📊 Average Scores:
Faithfulness: 1.00
Answer Relevance: 1.00
Context Relevance: 1.00


NameError: name 'plot_judge_scores' is not defined